In [1]:
import pandas as pd
import googlemaps
import os
from dotenv import load_dotenv, find_dotenv

In [19]:
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()
# load up the entries as environment variables
load_dotenv(dotenv_path)
# get the path of data
path_interim_data = os.environ.get("PATH_INTERIM_DATA_CLEANED")
path_processed_data = os.environ.get("PATH_PROCESSED_DATA")
# get google maps key
gmaps_key = os.environ.get("GOOGLE_MAPS_KEY")

# set filename
filename = 'restaurantes.csv'

# set country 
country = 'BR'

# set city name
city = 'São Carlos'

In [20]:
# load data
path_data = path_interim_data + filename
data = pd.read_csv(path_data)
data.head()

,Endereço
0,R MIGUEL PETRONI nº 1278 JARDIM BANDEIRAN...
1,R RAY WESLEY HERRICK nº 000915 JARDIM JOC...
2,R AQUIDABAN nº 000734 CENTRO CEP 13560-1...
3,R SETE DE SETEMBRO nº 001981 VILA NERY C...
4,AV SAO CARLOS nº 001609 CENTRO CEP 13560...


In [21]:
gmaps = googlemaps.Client(key=gmaps_key, queries_per_second=40)

components = {
    'country': country,
    'administrative_area': city
}

In [22]:
no_results = []
more_than_one = []
partial_match = []

for index, row in data.iterrows():
    try:
            
            address = row.Endereço
            result = gmaps.geocode(language='pt-BR', address= address, components=components)
            if not result:
                    print('No results')
                    no_results.append(row)
                    continue
            elif len(result) > 1:
                    print('More than one')
                    more_than_one.append(row)
                    continue
            elif result[0].get('partial_match') != None:
                    print('Partial Match')
                    partial_match.append(row)
                    continue
            else:
                    data.loc[index, 'LATITUDE'] = result[0]['geometry']['location']['lat']
                    data.loc[index, 'LONGITUDE'] = result[0]['geometry']['location']['lng']

    except Exception as e:
        print(e)

Partial Match
Partial Match
Partial Match
Partial Match
Partial Match
More than one
More than one
More than one
Partial Match
Partial Match
Partial Match
More than one
Partial Match
More than one
More than one
Partial Match
More than one
More than one
More than one
Partial Match
More than one
Partial Match
More than one
More than one
Partial Match
Partial Match
Partial Match
Partial Match
Partial Match
Partial Match
Partial Match
Partial Match
Partial Match
More than one
Partial Match
Partial Match
Partial Match
More than one
Partial Match
Partial Match
More than one
More than one
More than one
More than one
Partial Match
Partial Match
Partial Match
More than one
Partial Match
More than one
More than one
More than one
More than one
Partial Match
Partial Match
Partial Match
More than one
More than one
More than one
Partial Match
More than one
More than one
More than one
Partial Match
More than one
More than one
More than one


In [23]:
#save file
data.dropna(inplace=True)
data.to_csv(path_processed_data+filename, index=False)
establishmente_missing_references = pd.DataFrame(more_than_one+partial_match)
establishmente_missing_references.to_csv(path_interim_data+'restaurantes_missing_georeference.csv', index=False)